In [12]:
%reload_ext autoreload
%autoreload 2

In [13]:
from result_saver import SaverProvider

provider = SaverProvider()

# Parameters

In [14]:
# Fixed parameters

DEVICE = 'ibm_torino'
_RESETS = False

backend = provider.get_backend(DEVICE)

In [15]:
# Variable parameters
import numpy as np

logicals = ['0', '1']
x_basis = [True, False]

rounds = [10, 20, 30, 40, 50, 75, 100]

print("rounds:", rounds)

rounds: [10, 20, 30, 40, 50, 75, 100]


# Find the best layout

In [16]:
from soft_info import BackendEvaluator

evaluator = BackendEvaluator(backend)
longest_path, _, _, path_info = evaluator.find_longest_good_RepCode_string(longest_length=111)

Trying RepCode string of length 111 => distance 56... Current maxs: CX: 1.0, Ancilla: 1.0


In [17]:
path_info

{'mean_gate_error': 0.006532027953773052,
 'min_gate_error': 0.0014737259260892699,
 'max_gate_error': 0.03375907598632838,
 'mean_readout_error': 0.0275981981981982,
 'min_readout_error': 0.005900000000000016,
 'max_readout_error': 0.1582,
 'mean_ancilla_error': 0.03555636363636364,
 'min_ancilla_error': 0.011300000000000088,
 'max_ancilla_error': 0.1139,
 'CX_fidelity': 0.48543174121557786}

In [18]:
link_qubits = longest_path[1::2]
code_qubits = longest_path[0::2]
layout = link_qubits + code_qubits
distance = len(code_qubits)
print("longest path:", longest_path)
print("layout:", layout)
print("distance:", distance)

longest path: [129, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 113, 109, 108, 107, 94, 88, 89, 90, 75, 71, 70, 69, 56, 50, 51, 52, 37, 33, 32, 31, 18, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0, 15, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 36, 48, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 53, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 74, 86, 85, 84, 83, 82, 81, 80, 79, 78, 77, 76, 91, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106]
layout: [114, 116, 118, 120, 122, 124, 126, 128, 109, 107, 88, 90, 71, 69, 50, 52, 33, 31, 12, 10, 8, 6, 4, 2, 0, 19, 21, 23, 25, 27, 29, 48, 46, 44, 42, 40, 38, 57, 59, 61, 63, 65, 67, 86, 84, 82, 80, 78, 76, 95, 97, 99, 101, 103, 105, 129, 115, 117, 119, 121, 123, 125, 127, 113, 108, 94, 89, 75, 70, 56, 51, 37, 32, 18, 11, 9, 7, 5, 3, 1, 15, 20, 22, 24, 26, 28, 36, 47, 45, 43, 41, 39, 53, 58, 60, 62, 64, 66, 74, 85, 83, 81, 79, 77, 91, 96, 98, 100, 102, 104, 106]
distance: 56


# No rounds loop

In [19]:
from qiskit_qec.circuits import RepetitionCodeCircuit
from soft_info import run_IQ_calibration
from qiskit import transpile
from Scratch import metadata_helper
from tqdm import tqdm


############################

# Round index
round_index = 6

# Z = 0
xbasis = False
logical = '0'

############################

T = rounds[round_index]
print("Submitting for rounds = ", T)
REPETITIONS = 20

code = RepetitionCodeCircuit(distance, T, xbasis=xbasis)
qc = code.circuit[logical]

# Add repdelay at the end of the circuit 
range_rd = backend.configuration().rep_delay_range
qc.barrier()
qc.delay(unit='s', duration=4*range_rd[1])
qc.barrier()

# Transpile and run
transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=1) 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job
metadata = metadata_helper(descr='subset RepCodes', code="RepetitionCodeCircuit", 
                            distance=distance, rounds=T, logical=logical, xbasis=xbasis,
                            path_info=path_info)

# Run calibration
calib_shots = int(4e6/backend.configuration().n_qubits)
run_IQ_calibration(backend, shots=calib_shots)


for _ in tqdm(range(REPETITIONS)):
        backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single', 
                job_tags=[f'Subset {distance}, {T} rounds, {logical} log, xbasis={xbasis}'])
        
# Run calibration
run_IQ_calibration(backend, shots=calib_shots)


Submitting for rounds =  100


100%|██████████| 20/20 [04:25<00:00, 13.26s/it]


In [20]:
############################

# Z = 1
xbasis = False
logical = '1'

############################

T = rounds[round_index]
print("Submitting for rounds = ", T)
REPETITIONS = 20

code = RepetitionCodeCircuit(distance, T, xbasis=xbasis)
qc = code.circuit[logical]

# Add repdelay at the end of the circuit 
range_rd = backend.configuration().rep_delay_range
qc.barrier()
qc.delay(unit='s', duration=4*range_rd[1])
qc.barrier()

# Transpile and run
transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=1) 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job
metadata = metadata_helper(descr='subset RepCodes', code="RepetitionCodeCircuit", 
                            distance=distance, rounds=T, logical=logical, xbasis=xbasis,
                            path_info=path_info)

# Run calibration
calib_shots = int(4e6/backend.configuration().n_qubits)
run_IQ_calibration(backend, shots=calib_shots)


for _ in tqdm(range(REPETITIONS)):
        backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single', 
                job_tags=[f'Subset {distance}, {T} rounds, {logical} log, xbasis={xbasis}'])
        
# Run calibration
run_IQ_calibration(backend, shots=calib_shots)

Submitting for rounds =  100


100%|██████████| 20/20 [04:22<00:00, 13.10s/it]


In [21]:
############################

# X = 0
xbasis = True
logical = '0'

############################

T = rounds[round_index]
print("Submitting for rounds = ", T)
REPETITIONS = 20

code = RepetitionCodeCircuit(distance, T, xbasis=xbasis)
qc = code.circuit[logical]

# Add repdelay at the end of the circuit 
range_rd = backend.configuration().rep_delay_range
qc.barrier()
qc.delay(unit='s', duration=4*range_rd[1])
qc.barrier()

# Transpile and run
transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=1) 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job
metadata = metadata_helper(descr='subset RepCodes', code="RepetitionCodeCircuit", 
                            distance=distance, rounds=T, logical=logical, xbasis=xbasis,
                            path_info=path_info)

# Run calibration
calib_shots = int(4e6/backend.configuration().n_qubits)
run_IQ_calibration(backend, shots=calib_shots)


for _ in tqdm(range(REPETITIONS)):
        backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single', 
                job_tags=[f'Subset {distance}, {T} rounds, {logical} log, xbasis={xbasis}'])
        
# Run calibration
run_IQ_calibration(backend, shots=calib_shots)

Submitting for rounds =  100


100%|██████████| 20/20 [04:32<00:00, 13.62s/it]


In [22]:
############################

# X = 1
xbasis = True
logical = '1'

############################

T = rounds[round_index]
print("Submitting for rounds = ", T)
REPETITIONS = 20

code = RepetitionCodeCircuit(distance, T, xbasis=xbasis)
qc = code.circuit[logical]

# Add repdelay at the end of the circuit 
range_rd = backend.configuration().rep_delay_range
qc.barrier()
qc.delay(unit='s', duration=4*range_rd[1])
qc.barrier()

# Transpile and run
transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=1) 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job
metadata = metadata_helper(descr='subset RepCodes', code="RepetitionCodeCircuit", 
                            distance=distance, rounds=T, logical=logical, xbasis=xbasis,
                            path_info=path_info)

# Run calibration
calib_shots = int(4e6/backend.configuration().n_qubits)
run_IQ_calibration(backend, shots=calib_shots)


for _ in tqdm(range(REPETITIONS)):
        backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single', 
                job_tags=[f'Subset {distance}, {T} rounds, {logical} log, xbasis={xbasis}'])
        
# Run calibration
run_IQ_calibration(backend, shots=calib_shots)

Submitting for rounds =  100


100%|██████████| 20/20 [52:34<00:00, 157.72s/it]


KeyboardInterrupt: 

# Round loop!

In [30]:

# Z = 0
xbasis = False
logical = '0'

# Run calibration

for T in rounds:
    code = RepetitionCodeCircuit(distance, T, xbasis=xbasis)
    qc = code.circuit[logical]
    transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=1) 

    n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job
    metadata = metadata_helper(descr='subset RepCodes', code="RepetitionCodeCircuit", 
                               distance=distance, rounds=T, logical=logical, xbasis=xbasis,
                               path_info=path_info)

    # backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single', 
    #             job_tags=[f'Subset {distance}, {T} rounds, {logical} log, xbasis={xbasis}'])

# Run calibration
    

{'descr': 'subset RepCodes', 'code': 'RepetitionCodeCircuit', 'distance': 52, 'rounds': 10, 'logical': '0', 'xbasis': False, 'path_info': {'mean_gate_error': 0.007963147793478936, 'min_gate_error': 0.0034766228550469347, 'max_gate_error': 0.0232547718972346, 'mean_readout_error': 0.022770873786407766, 'min_readout_error': 0.0019000000000000128, 'max_readout_error': 0.3568, 'mean_ancilla_error': 0.02186078431372549, 'min_ancilla_error': 0.0019000000000000128, 'max_ancilla_error': 0.24750000000000005}}
{'descr': 'subset RepCodes', 'code': 'RepetitionCodeCircuit', 'distance': 52, 'rounds': 20, 'logical': '0', 'xbasis': False, 'path_info': {'mean_gate_error': 0.007963147793478936, 'min_gate_error': 0.0034766228550469347, 'max_gate_error': 0.0232547718972346, 'mean_readout_error': 0.022770873786407766, 'min_readout_error': 0.0019000000000000128, 'max_readout_error': 0.3568, 'mean_ancilla_error': 0.02186078431372549, 'min_ancilla_error': 0.0019000000000000128, 'max_ancilla_error': 0.24750000